In [1]:
from zabbix_api import ZabbixAPI
from datetime import datetime
import time
import pandas as pd

In [2]:
zapi = ZabbixAPI("https://sm-fe01.webmonitor.global")
# Disable SSL certificate verification
zapi.session.verify = False
# Specify a timeout (in seconds)
zapi.timeout = 10.1
# informacoes de acesso
zapi.login("linx.rafael", "stk456rfs")
# mostra versao do zabbix
print("Connected to Zabbix API Version %s" % zapi.api_version())

Connected to Zabbix API Version 3.4.10


In [3]:
import mysql.connector

from mysql.connector import errorcode 

# obtendo conection string 
config = {
    'host':'192.168.121.246',
    'user':'API',
    'password':'stk456rfs',
    'database':'novokpi'
}

In [4]:
# Construct connection string
try:
   conn = mysql.connector.connect(**config)
   df = pd.read_sql('select * from tb_item', conn)
   print("Connection established")
except mysql.connector.Error as err:
  if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
    print("Something is wrong with the user name or password")
  elif err.errno == errorcode.ER_BAD_DB_ERROR:
    print("Database does not exist")
  else:
    print(err)
else:
  cursor = conn.cursor()

Connection established


In [5]:
# Criar o intervalo de coleta 
time_from = time.mktime(datetime.now().timetuple()) - 60 * 60 * 24  # 1 dia (inteiro)
print(time_from)

1547050599.0


In [6]:
count_row = df.shape[0]  # gives number of row count
count_col = df.shape[1]  # gives number of col count

print(count_row)
print(count_col)
print(df)

109
5
     item_id  item_id_zabbix item_produto  \
0          1         2212476         MIDe   
1          2         2212479         MIDe   
2          3         2212481         MIDe   
3          4         2212483         MIDe   
4          5         2198455         MIDe   
5          6         2198498         MIDe   
6          7         2199995         MIDe   
7          8         2198369         MIDe   
8          9         2198412         MIDe   
9         10         2003427         DTEF   
10        11          917847     Microvix   
11        12         2156697     Microvix   
12        13         2184022     Microvix   
13        14          878233     Microvix   
14        15          878234     Microvix   
15        16          878235     Microvix   
16        17          878236     Microvix   
17        18          878237     Microvix   
18        19          878238     Microvix   
19        20          878239     Microvix   
20        21          878240     Microvix   
21  

In [7]:
n = 0
count_row = df.shape[0] 


while  n < count_row:
    
    item_id = df.loc[n, 'item_id']
    print(item_id)
    
    item_id_zabbix = df['item_id_zabbix'].loc[n]
    print(item_id_zabbix)
    
    item_produto = df['item_produto'].loc[n]
    print(item_produto)
    
    
    historico = zapi.history.get({ 'itemids': [ str(item_id_zabbix) ], 
    'history': 0, 
    'output': 'extend', 
    'time_from': time_from, 
    }) 

    for point in historico:       # Ver uma possibilidade logica de colocar um looping aqui pois alguns dados podem ter mais de uma amostra no mesmo periodo isso é um saco!!!!
        # tem que converter o dado da API para manter em variavel
        dtItem = ("{0}".format(datetime.fromtimestamp(int(point['clock'])).strftime("%x %X"), point['value']))
        valorItem = ('{1}'.format(datetime.fromtimestamp(int(point['clock'])).strftime("%x %X"), point['value']))
        
    print(dtItem)

    #vi = valorItem.replace('.' , ',')

    vi = float(valorItem)
    print(vi)
        
    # Inserir os dados na tabela (preferivel já ter mapeada)
    cursor.execute("INSERT INTO tb_kpi(tb_item_item_id, kpi_valor, kpi_dttime) VALUES('" + str(item_id) + "','" + str(vi) + "','" + str(dtItem) + "')")
    print("Inserted",cursor.rowcount,"row(s) of data.")
    

    print('----------------------------------------------------')
    n = n + 1

1
2212476
MIDe


NameError: name 'dtItem' is not defined

In [ ]:
# Cleanup

conn.commit()
cursor.close()
conn.close()

print("Connection finished")
print("Done.")